## Using TextFile

<br>Get the consolidated account statement  from CAMS
<br>Remove  Password  Protection using print to PDF
<br>Open with PHantom Foxit PDF or Chrome and do  select all, copy and paste to text file

In [1]:
import pandas as pd
import logging

import src.constants as ct
from src.parsedata import DataParser
from src.initialize import Logger

pd.options.display.max_columns = 100
# pd.options.display.max_rows = 100
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)



## Read and Parse MF Transaction Data from text file

In [2]:
logger = Logger()
logger.initilaze_logging(display_logs = 'yes') 

In [3]:
file_path = 'data\consol_trans_18_June_2021.txt'
dp = DataParser()
mf_trans_df, mf_hdr_df = dp.parse_mf_data(file_path)

File data\consol_trans_18_June_2021.txt sucessfully read, found 505 records



In [4]:
mf_trans_df[-10:]

,scheme_name,trans_date,amount,units,unit_price
216,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2021-01-21,24998.75,310.412,80.534
217,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2021-02-22,24998.75,298.403,83.775
218,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2021-03-21,24998.75,294.599,84.857
219,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2021-04-22,24998.75,299.447,83.483
220,Mirae Asset Emerging Bluechip Fund-Direct Plan-Growth,2021-05-21,24998.75,275.742,90.660
221,Mirae Asset Large Cap Fund-Direct Plan-Growth,2019-02-19,250000.00,5098.607,49.033
222,Mirae Asset Large Cap Fund-Direct Plan-Growth,2019-06-10,150000.00,2708.119,55.389
223,Mirae Asset Large Cap Fund-Direct Plan-Growth,2019-06-14,300000.00,5464.680,54.898
224,Mirae Asset Large Cap Fund-Direct Plan-Growth,2020-03-17,200000.00,4548.452,43.971
225,Mirae Asset Tax Saver Fund-Direct Growth,2020-12-18,59997.00,2456.678,24.422


In [5]:
mf_hdr_df

,scheme_name,latest_nav,latest_nav_date
0,DSP Small Cap Fund-Direct Plan-Growth,98.2220,2021-06-17
1,Franklin India Flexi Cap Fund-Direct-Growth,903.5355,2021-06-17
2,Franklin India Smaller Companies Fund-Direct-Growth,83.3396,2021-06-17
3,Franklin India Ultra Short Bond Fund Super Institutional Plan-Direct-Growth,30.2922,2021-06-17
4,Franklin India Low Duration Fund-Direct-Growth,24.2658,2021-06-17
5,HDFC Top 100 Fund-Direct Plan-Growth Option,661.4190,2021-06-17
6,ICICI Prudential All Seasons Bond Fund-Direct Plan-Growth,29.7814,2021-06-17
7,ICICI Prudential Banking and PSU Debt Fund-Direct Plan-Growth,26.0015,2021-06-17
8,ICICI Prudential Long Term Equity Fund-Direct Plan-Growth,562.5800,2021-06-17
9,ICICI Prudential Nifty Next 50 Index Fund-Direct Plan-Growth,35.1068,2021-06-17


## Calculate MF Capital Gains

In [6]:

from src.capitalgains import CapitalGains
target_ltcg = 100000
cg = CapitalGains(mf_hdr_df, mf_trans_df)
cg.prepare_final_data(target_ltcg)

Schemes below  are removed as  Grandfathered NAV was not Found
Franklin India Ultra Short Bond Fund Super Institutional Plan-Direct-Growth
Mirae Asset Tax Saver Fund-Direct Growth
ICICI Prudential Long Term Equity Fund-Direct Plan-Growth
No Missing data Found:
For Scheme DSP Small Cap Fund-Direct Plan-Growth: Sell 3563.92 units for LTCG of 100000.0
For Scheme Franklin India Flexi Cap Fund-Direct-Growth: Sell 376.05 units for LTCG of 100000.0
For Scheme Franklin India Smaller Companies Fund-Direct-Growth: Sell 5582.73 units for LTCG of 100000.0
For Scheme Franklin India Low Duration Fund-Direct-Growth Total LTCG 68423.93 is less than target LTCG 100000: Sell 15952.61 units for LTCG of 68423.93
For Scheme HDFC Top 100 Fund-Direct Plan-Growth Option: Sell 612.03 units for LTCG of 100000.0
For Scheme ICICI Prudential All Seasons Bond Fund-Direct Plan-Growth: Sell 12587.17 units for LTCG of 100000.0
For Scheme ICICI Prudential Banking and PSU Debt Fund-Direct Plan-Growth: Sell 16322.26 unit

In [7]:
cg.output_hdr_df

,scheme_name,latest_nav,latest_nav_date,gf_nav,amount,units,ltcg,stcg,current_amt,target_ltcg,target_units,comments,perc_gain
0,DSP Small Cap Fund-Direct Plan-Growth,98.2220,17-Jun-2021,70.1630,759990.00,13196.649,295479.48,61869.86,1296201.26,100000.00,3563.917,Sell 3563.92 units for LTCG of 100000.0,70.555042
1,Franklin India Flexi Cap Fund-Direct-Growth,903.5355,17-Jun-2021,637.6120,905000.00,1643.183,476079.13,0.00,1484674.22,100000.00,376.051,Sell 376.05 units for LTCG of 100000.0,64.052400
2,Franklin India Smaller Companies Fund-Direct-Growth,83.3396,17-Jun-2021,65.5900,697000.00,12301.352,291192.24,13758.60,1025189.77,100000.00,5582.729,Sell 5582.73 units for LTCG of 100000.0,47.086050
3,Franklin India Low Duration Fund-Direct-Growth,24.2658,17-Jun-2021,19.9766,300000.00,15952.610,68423.93,0.00,387102.84,68423.93,15952.610,Total LTCG 68423.93 is less than target LTCG 100000: Sell 15952.61 units for LTCG of 68423.93,29.034280
4,HDFC Top 100 Fund-Direct Plan-Growth Option,661.4190,17-Jun-2021,498.0290,569833.57,1331.043,219706.64,0.00,880377.13,100000.00,612.031,Sell 612.03 units for LTCG of 100000.0,54.497239
5,ICICI Prudential All Seasons Bond Fund-Direct Plan-Growth,29.7814,17-Jun-2021,21.8368,699980.00,29838.991,129766.37,2220.24,888646.93,100000.00,12587.168,Sell 12587.17 units for LTCG of 100000.0,26.953189
6,ICICI Prudential Banking and PSU Debt Fund-Direct Plan-Growth,26.0015,17-Jun-2021,19.8749,799975.00,36754.760,106882.29,2091.77,955678.90,100000.00,16322.264,Sell 16322.26 units for LTCG of 100000.0,19.463596
7,ICICI Prudential Nifty Next 50 Index Fund-Direct Plan-Growth,35.1068,17-Jun-2021,27.5101,577500.00,22713.467,221485.09,0.00,797397.13,100000.00,11887.388,Sell 11887.39 units for LTCG of 100000.0,38.077425
8,ICICI Prudential US Bluechip Equity Fund-Direct Plan-Growth,47.9900,17-Jun-2021,25.0600,574971.25,14022.847,0.00,97985.15,672956.43,0.00,0.000,No LTCG exists hence calculation is not applicable,17.041753
9,ICICI Prudential Value Discovery Fund-Direct Plan-Growth,230.3200,17-Jun-2021,157.6400,944980.75,6237.578,309577.68,89076.57,1436638.97,100000.00,1375.894,Sell 1375.89 units for LTCG of 100000.0,52.028385


In [8]:
cg.output_trans_df.head()

,scheme_name,trans_date,amount,units,unit_price,latest_nav,latest_nav_date,gf_nav,new_purch_nav,ltcg,stcg,cumil_ltcg,cumil_units,current_amt,perc_gain
0,DSP Small Cap Fund-Direct Plan-Growth,05-Jan-2017,60000.0,1141.770,52.550,98.222,17-Jun-2021,70.163,70.163,32036.92,0.0,32036.92,1141.77,112146.93,86.91
1,DSP Small Cap Fund-Direct Plan-Growth,10-Jan-2017,100000.0,1895.842,52.747,98.222,17-Jun-2021,70.163,70.163,53195.43,0.0,85232.36,3037.61,186213.39,86.21
2,DSP Small Cap Fund-Direct Plan-Growth,12-Jan-2017,100000.0,1877.511,53.262,98.222,17-Jun-2021,70.163,70.163,52681.08,0.0,137913.44,4915.12,184412.89,84.41
3,DSP Small Cap Fund-Direct Plan-Growth,13-Jan-2017,100000.0,1880.795,53.169,98.222,17-Jun-2021,70.163,70.163,52773.23,0.0,190686.66,6795.92,184735.45,84.74
4,DSP Small Cap Fund-Direct Plan-Growth,16-Jan-2017,100000.0,1875.891,53.308,98.222,17-Jun-2021,70.163,70.163,52635.63,0.0,243322.29,8671.81,184253.77,84.25
